In [1]:
# Intall and Import Package/Library
!pip install pulp
import pulp
!apt-get install -y -qq glpk-utils
from pulp import GLPK

# Library to handle data
import pandas as pd
import numpy as np


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 63.5 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking 

In [2]:
data = pd.read_excel("processed_data_F.xlsx")
columns_to_normalize = data.columns[1:12]


for column in columns_to_normalize:

    max_value = data[column].max()

    data[column] = data[column] / max_value
data.head()

,player,hoursplayed,minsplayed,age,money_peryear,1/yellow_card,yellowcard,goal,assisttotal,shots_pergame,pass_success_percent,pass_success,position
0,Erling Haaland,0.792997,0.792997,0.741935,1.000000,1.000000,0.090909,1.000000,0.384615,1.000000,0.865772,0.865772,F
1,Mohamed Salah,0.785869,0.785869,1.000000,0.933333,0.500000,0.181818,0.666667,0.769231,0.923077,0.846756,0.846756,F
2,Bernardo Silva,0.800124,0.800124,0.935484,0.800000,0.125000,0.727273,0.222222,0.692308,0.256410,1.000000,1.000000,F
3,Jack Grealish,0.311125,0.311125,0.903226,0.800000,0.142857,0.636364,0.111111,0.076923,0.282051,0.980984,0.980984,F
4,Marcus Rashford,0.706229,0.706229,0.838710,0.800000,0.500000,0.181818,0.259259,0.153846,0.487179,0.885906,0.885906,F


In [4]:
for k in range(20):  # we expect to solve 20 LP problems to get the most favourable relative efficiency score for each player
    model = pulp.LpProblem("football player Analytics", pulp.LpMaximize) # Create an LP maximization problem
    delta = pulp.LpVariable("delta",  lowBound=0, upBound=None, cat='Continuous')
    m1 = pulp.LpVariable("m1", lowBound=0, upBound=None, cat='Continuous') # Create a variable m1>=0
    m2 = pulp.LpVariable("m2", lowBound=0, upBound=None, cat='Continuous') # Create a variable m2>=0
    m3 = pulp.LpVariable("m3", lowBound=0, upBound=None, cat='Continuous') # Create a variable m3>=0
    m4 = pulp.LpVariable("m4", lowBound=0, upBound=None, cat='Continuous') # Create a variable m4>=0
    n1 = pulp.LpVariable("n1", lowBound=0, upBound=None, cat='Continuous') # Create a variable n1>=0
    n2 = pulp.LpVariable("n2", lowBound=0, upBound=None, cat='Continuous') # Create a variable n2>=0
    n3 = pulp.LpVariable("n3", lowBound=0, upBound=None, cat='Continuous') # Create a variable n3>=0

    scale = pulp.LpVariable("scale", lowBound=0, upBound=None, cat='Continuous')


    model += delta
    model += (m1*data["goal"][k] + m2*data["assisttotal"][k] + m3*data["shots_pergame"][k] + m4*data["pass_success_percent"][k]) == delta  #objective function

    model += n1*data["hoursplayed"][k] + n2*data["age"][k] + n3*data["money_peryear"][k]  == 1 #first constraint (equality)

    for i in range(20): # loop for 20 times for the next 20 constraints
        model += m1*data["goal"][i] + m2*data["assisttotal"][i] + m3*data["shots_pergame"][i] + m4*data["pass_success_percent"][i] - n1*data["hoursplayed"][i] - n2*data["age"][i] - n3*data["money_peryear"][i] <= 0

        model += scale >= (m1 * data["goal"][i] +
                            m2 * data["assisttotal"][i] +
                            m3 * data["shots_pergame"][i] +
                            m4 * data["pass_success_percent"][i])

    status = model.solve(GLPK(msg=True, options=['--ranges', 'sensitivity.txt'])) # Solve the Problem and get the sensitivity analysis report
    print("Model Status:{}".format(pulp.LpStatus[model.status])) # Print the status of the solution (e.g., optimal, infeasible, unbounded)

    # To Display Optimal Decision Variables & Reduced Cost per Variable
    for v in model.variables():
	      print(v.name, "=", v.varValue, "\tReduced Cost =", v.dj)

    # To Display Optimal Value of Objective Function
    #each objective value represents efficiency score of each player.

    print("Objective=", pulp.value(model.objective))

    # To Display Shadow Price & Slack Value per Constraint
    print ("\nSensitivity Analysis\nConstraint\t\tShadow Price\tSlack")
    for name, c in model.constraints.items():
	      print(name, ":", c, "\t", c.pi, "\t\t", c.slack)


Model Status:Optimal
delta = 1.0 	Reduced Cost = None
m1 = 1.0 	Reduced Cost = None
m2 = 0.0 	Reduced Cost = None
m3 = 0.0 	Reduced Cost = None
m4 = 0.0 	Reduced Cost = None
n1 = 0.492401 	Reduced Cost = None
n2 = 0.0 	Reduced Cost = None
n3 = 0.609528 	Reduced Cost = None
scale = 1.0 	Reduced Cost = None
Objective= 1.0

Sensitivity Analysis
Constraint		Shadow Price	Slack
_C1 : -delta + m1 + 0.38461538461538464*m2 + m3 + 0.8657718120805369*m4 = -0.0 	 None 		 None
_C2 : 0.7929965912612333*n1 + 0.7419354838709677*n2 + n3 = 1.0 	 None 		 None
_C3 : m1 + 0.38461538461538464*m2 + m3 + 0.8657718120805369*m4 - 0.7929965912612333*n1 - 0.7419354838709677*n2 - n3 <= -0.0 	 None 		 None
_C4 : -m1 - 0.38461538461538464*m2 - m3 - 0.8657718120805369*m4 + scale >= -0.0 	 None 		 None
_C5 : 0.6666666666666666*m1 + 0.7692307692307693*m2 + 0.9230769230769231*m3 + 0.8467561521252797*m4 - 0.7858692283854973*n1 - n2 - 0.9333333333333333*n3 <= -0.0 	 None 		 None
_C6 : -0.6666666666666666*m1 - 0.7692307692